In [1]:
import os
import numpy as np
import torch
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import math
import torch.nn as nn
import os
import numpy as np
import argparse
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import copy
import math
from tqdm import tqdm, trange
from torch.nn import SyncBatchNorm
from torch.utils.data.dataloader import default_collate
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
# from torch.utils.tensorboard import SummaryWriter
import datasets
from datasets import load_dataset
# import evaluate
work_dir = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

In [3]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
dataset

Using the latest cached version of the module from /home/adminniu/.cache/huggingface/modules/datasets_modules/datasets/cnn_dailymail/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de (last modified on Fri Mar 10 11:32:23 2023) since it couldn't be found locally at cnn_dailymail., or remotely on the Hugging Face Hub.
Found cached dataset cnn_dailymail (/home/adminniu/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
input_ids = tokenizer(dataset["train"][0]["article"], return_tensors="pt")
input_ids

{'input_ids': tensor([[250004,      6, 106699,  89539,      4,  30715,     15, 192619,     16,
           4210,  20904,  59566,   6057,  11555,   9552, 195167,     13,  21647,
              7,  17203,     47,     10, 113771,  11762,   1549,  19879,     15,
           4369,    617,  92134,  19879,     16,    100,  45091,    237,    764,
          15504,      7,    543,     98,  68318,      4,   1284,    764,  51636,
              7,     70,  17265,  23742,     25,     18,  37702,     10, 160093,
             98,   4049,      5,  11555,   9552, 195167,     13,    237,  20904,
          59566,     23,     44,  30404,   1294,  59566,    136,     70,  81206,
            111,     70, 133720,     58,    717,     70,   2837, 140545,    674,
            111, 207039,   3365,    316,  42729,      7,  10932,     70,   8999,
              4,     70,  27150,  39329,  17378,    764,   1556,    110,  38627,
             47,  97613,     56,   1919,  49456,  16065,     98,   4271,   2258,
              

In [5]:
output = model.generate(**input_ids)
output

/home/adminniu/anaconda3/envs/lhr/lib/python3.7/site-packages/transformers/generation/utils.py:1278: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


tensor([[     2,      0,  11555,   9552, 195167,     13,    237,  20904,  59566,
             23,     44,  30404,   1294,  59566,    136,     70,  81206,    111,
             70, 133720,     58,      2]])

In [7]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix"'

In [41]:
print(dataset["train"][0]["article"])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Trainer, TrainingArguments
from transformers.models.bart import shift_tokens_right


def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['text'], pad_to_max_length=True, max_length=1024, truncation=True)
    target_encodings = tokenizer.batch_encode_plus(example_batch['summary'], pad_to_max_length=True, max_length=1024, truncation=True)
    
    labels = target_encodings['input_ids']
    decoder_input_ids = shift_tokens_right(labels, model.config.pad_token_id)
    labels[labels[:, :] == model.config.pad_token_id] = -100
    
    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': decoder_input_ids,
        'labels': labels,
    }

    return encodings

dataset = dataset.map(convert_to_features, batched=True)
columns = ['input_ids', 'labels', 'decoder_input_ids','attention_mask',] 
dataset.set_format(type='torch', columns=columns)

training_args = TrainingArguments(
    output_dir='./models/bart-summarizer',          
    num_train_epochs=1,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',          
)

trainer = Trainer(
    model=model,                       
    args=training_args,                  
    train_dataset=dataset['train'],        
    eval_dataset=dataset['validation']   
)

ImportError: cannot import name 'shift_tokens_right' from 'transformers.models.bart' (/home/adminniu/anaconda3/envs/lhr/lib/python3.7/site-packages/transformers/models/bart/__init__.py)

In [12]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  l



Packages:
Error connecting to server: [Errno 111] Connection refused

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [15]:
nltk.data.find("tokenizers/punkt")

FileSystemPathPointer('/home/adminniu/nltk_data/tokenizers/punkt/PY3')